In [5]:
import requests
from pymongo import MongoClient

In [6]:
# Client connects to "localhost" by default
client = MongoClient()

# Create local "nobel" database on the fly
db = client["nobel"]

for collection_name in ["prizes", "laureates"]:
# collect the data from the API
    response = requests.get("http://api.nobelprize.org/v1/{}.json".\
        format(collection_name[:-1] ))

    # convert the data to json
    documents = response.json()[collection_name]

    # Create collections on the fly
    db[collection_name].insert_many(documents)


In [7]:
# client is a dictionary of databases
db = client["nobel"]

# database is a dictionary of collections
prizes_collection = db["prizes"]


In [8]:
from collections import OrderedDict
from itertools import groupby
from operator import itemgetter

original_categories = set(db.prizes.distinct("category", {"year": "1901"}))

# Save an pipeline to collect original-category prizes
pipeline = [
    {"$match": {"category": {"$in": list(original_categories)}}},
    {"$project": {"year": 1, "category": 1}},
    {"$sort": OrderedDict([("year", -1)])}
]
cursor = db.prizes.aggregate(pipeline)
for key, group in groupby(cursor, key=itemgetter("year")):
    missing = original_categories - {doc["category"] for doc in group}
    if missing:
        print("{year}: {missing}".format(year=key, missing=", ".join(sorted(missing))))